In [1]:
#Import packages needed
import pandas as pd
import numpy as np
import gmaps
import requests
import re
from config import gkey
from pprint import pprint
gmaps.configure(api_key=gkey)
import matplotlib.pyplot as plt

In [2]:
# Bring in csv and look at data
# jobscat_csv = "../../Archive/completeAndCleanNycJobs_kh.csv"
jobscat_df = pd.read_csv("../../Archive/completeAndCleanNycJobs_kh.csv", dtype={'zip': 'str'})
jobscat_df.shape

(2965, 13)

In [12]:
# List out the columns in dataframe
list(jobscat_df)

['index',
 'agency',
 'numOfPositions',
 'businessTitle',
 'jobCategory',
 'newLat',
 'newLong',
 'borough',
 'salaryRangeTo',
 'salaryRangeFrom',
 'salaryFrequency',
 'zip',
 'jobCatCondensed']

In [29]:
# Groupby Borough and Category and change name of columns
boroughgroup = jobscat_df.groupby(["borough", "jobCatCondensed"])\
                                    .agg({'zip': "count", 
                                          "numOfPositions": "sum",
                                         'salaryRangeFrom': "min",
                                         'salaryRangeTo':"max"})
boroughgroup["salaryRangeFrom"] = boroughgroup["salaryRangeFrom"].map("${:,.2f}".format)
boroughgroup["salaryRangeTo"] = boroughgroup["salaryRangeTo"].map("${:,.2f}".format)

boroughgroup = boroughgroup.rename(columns={"borough": "Borough",
                                            "zip":"Number of Jobs",
                                    "numOfPositions":"Number of Positions/Job",
                                    "salaryRangeFrom": "Salary Minimun",
                                    "salaryRangeTo": "Salary Max"})

boroughgroup = boroughgroup.rename_axis(["Borough", "Category"])
boroughgroup

Number of Jobs  \
Borough       Category                                               
Brooklyn      Managerial and Professional                       46   
              Operatives and Laborers                           48   
              Other                                            249   
              Service                                           29   
Manhattan     Managerial and Professional                      167   
              Operatives and Laborers                           65   
              Other                                            901   
              Service                                          289   
              Technical, Sales, and Administrative              13   
Queens        Managerial and Professional                      188   
              Operatives and Laborers                           55   
              Other                                            827   
              Service                                           17   
              Technical, Sales, and Administrative               5   
Staten Island Managerial and Professional                        9   
              Operatives and Laborers                           16   
              Other                                             10   
The Bronx     Managerial and Professional                       14   
              Operatives and Laborers                            5   
              Other                                              7   
              Service                                            4   
              Technical, Sales, and Administrative               1   

                                                    Number of Positions/Job  \
Borough       Category                                                        
Brooklyn      Managerial and Professional                               254   
              Operatives and Laborers                                   413   
              Other                                                     754   
              Service                                                    46   
Manhattan     Managerial and Professional                               380   
              Operatives and Laborers                                   434   
              Other                                                    1791   
              Service                                                   542   
              Technical, Sales, and Administrative                       19   
Queens        Managerial and Professional                              1329   
              Operatives and Laborers                                   127   
              Other                                                    1149   
              Service                                                    63   
              Technical, Sales, and Administrative                        5   
Staten Island Managerial and Professional                                 9   
              Operatives and Laborers                                    68   
              Other                                                      12   
The Bronx     Managerial and Professional                                16   
              Operatives and Laborers                                     5   
              Other                                                      14   
              Service                                                     4   
              Technical, Sales, and Administrative                        1   

                                                   Salary Minimun   Salary Max  
Borough       Category                                                          
Brooklyn      Managerial and Professional                   $0.00  $140,000.00  
              Operatives and Laborers                      $16.14  $156,829.00  
              Other                                         $0.00  $225,217.00  
              Service                                      $28.00  $1

In [30]:
# Slicing the dataframe into smaller dataframes by borough

BrooklynGroup = (boroughgroup[boroughgroup.index.get_level_values("Borough").str.startswith('Brook')])
ManhattanGroup = (boroughgroup[boroughgroup.index.get_level_values("Borough").str.startswith('Man')])
QueensGroup = (boroughgroup[boroughgroup.index.get_level_values("Borough").str.startswith('Queens')])
StatenIslandGroup = (boroughgroup[boroughgroup.index.get_level_values("Borough").str.startswith('Staten')])
BronxGroup = (boroughgroup[boroughgroup.index.get_level_values("Borough").str.startswith('The Bronx')])

In [31]:
# Test to ses what one borough dataframe looks like
StatenIslandGroup

Number of Jobs  \
Borough       Category                                      
Staten Island Managerial and Professional               9   
              Operatives and Laborers                  16   
              Other                                    10   

                                           Number of Positions/Job  \
Borough       Category                                               
Staten Island Managerial and Professional                        9   
              Operatives and Laborers                           68   
              Other                                             12   

                                          Salary Minimun   Salary Max  
Borough       Category                                                 
Staten Island Managerial and Professional     $37,796.00   $89,942.00  
              Operatives and Laborers             $58.40  $135,000.00  
              Other                               $45.49  $150,371.00

In [32]:
# Save dataframe to HTML files

BrooklynGroup.to_html("BoroughHTMLFolder/Brooklyn.html")
ManhattanGroup.to_html("BoroughHTMLFolder/Manhattan.html")
QueensGroup.to_html("BoroughHTMLFolder/Queens.html")
StatenIslandGroup.to_html("BoroughHTMLFolder/StatenIsland.html")
BronxGroup.to_html("BoroughHTMLFolder/Bronx.html")